# Bilibili's Cost of Equity

### Import essential libraries

In [1]:
import pandas as pd
import numpy as np
import datetime as dt 
import statsmodels.api as sm
from pandas_datareader import DataReader as web

### Fetch daily stock prices from yahoo

In [2]:
start_date = dt.date(2018, 3, 28)
end_date = dt.date.today()

tickers = ['BILI', '^GSPC', '^TNX']    # Tickers for Bilibili, S&P500 and US 10-year treasury bond

In [3]:
df = pd.DataFrame()

for ticker in tickers:
    df[ticker] = web(ticker, 'yahoo', start_date, end_date)['Adj Close']

In [4]:
df.rename(columns={'^GSPC': 'S&P 500', '^TNX': '10-year T-bills'}, inplace=True)
df.head()

,BILI,S&P 500,10-year T-bills
Date,,,
2018-03-28,11.24,2605.000000,2.775
2018-03-29,11.00,2640.870117,2.741
2018-04-02,11.00,2581.879883,2.732
2018-04-03,10.91,2614.449951,2.784
2018-04-04,10.97,2644.689941,2.788


### Convert stock prices into daily returns

In [5]:
stock_returns = df.copy()
stock_returns.dropna(inplace=True)

for column in df.columns[:2]:
#     stock_returns[column] = np.log(stock_returns[column]) - np.log(stock_returns[column].shift(1))
    stock_returns[column] = stock_returns[column].pct_change()

stock_returns.head()

,BILI,S&P 500,10-year T-bills
Date,,,
2018-03-28,NaN,NaN,2.775
2018-03-29,-0.021352,0.013770,2.741
2018-04-02,0.000000,-0.022337,2.732
2018-04-03,-0.008182,0.012615,2.784
2018-04-04,0.005500,0.011566,2.788


### Presenting OLS estimation of beta

In [6]:
risk_free = stock_returns['10-year T-bills'].mean() / 100    # Average bond yield

stock_returns.dropna(inplace=True)
stock_returns['MRP'] = stock_returns['S&P 500'] - risk_free
stock_returns.head()

,BILI,S&P 500,10-year T-bills,MRP
Date,,,,
2018-03-29,-0.021352,0.013770,2.741,-0.004890
2018-04-02,0.000000,-0.022337,2.732,-0.040997
2018-04-03,-0.008182,0.012615,2.784,-0.006045
2018-04-04,0.005500,0.011566,2.788,-0.007093
2018-04-05,0.000912,0.006863,2.832,-0.011797


In [7]:
X = sm.add_constant(stock_returns['MRP'])
y = stock_returns['BILI']

reg_model = sm.OLS(y, X)
results = reg_model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   BILI   R-squared:                       0.121
Model:                            OLS   Adj. R-squared:                  0.120
Method:                 Least Squares   F-statistic:                     100.8
Date:                Sat, 06 Mar 2021   Prob (F-statistic):           2.62e-22
Time:                        18:53:33   Log-Likelihood:                 1355.9
No. Observations:                 731   AIC:                            -2708.
Df Residuals:                     729   BIC:                            -2699.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0214      0.002      9.614      0.000       0.017       0.026
MRP            0.9643      0.096     10.041      0.000       0.776       1.153
==============================================================================
Omnibus:                       88.383   Durbin-Watson:                   2.137
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              271.041
Skew:                           0.580   Prob(JB):                     1.39e-59
Kurtosis:                       5.748   Cond. No.                         68.5
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [8]:
beta = results.params['MRP']
beta

0.9643434358381104

### Using CAPM to calculate the cost of equity

In [9]:
market_return = stock_returns['S&P 500'].mean()    # obtain expected return for market portfolio

re = risk_free + beta * (market_return - risk_free)
re

0.0012816643100163963

In [10]:
market_return

0.0006391146064095803

In [11]:
risk_free

0.018659631128682467

In [12]:
stock_returns[stock_returns['S&P 500'] < 0.0005]

,BILI,S&P 500,10-year T-bills,MRP
Date,,,,
2018-04-02,0.000000,-0.022337,2.732,-0.040997
2018-04-06,-0.003643,-0.021920,2.775,-0.040580
2018-04-11,-0.001739,-0.005525,2.790,-0.024185
2018-04-13,-0.018933,-0.002887,2.829,-0.021546
2018-04-19,0.000000,-0.005726,2.914,-0.024386
...,...,...,...,...
2021-02-25,0.036967,-0.024479,1.518,-0.043138
2021-02-26,-0.044524,-0.004750,1.460,-0.023410
2021-03-02,-0.046520,-0.008081,1.415,-0.026740
